# Exercício 1 – Teórico: Fundamentos de RAG

Explique, com suas próprias palavras:

1. O que é RAG (Retrieval-Augmented Generation) e qual problema ele resolve na geração de texto usando modelos de linguagem?
2. Quais são as vantagens de utilizar RAG em vez de depender apenas do modelo base?
3. Quais são as principais etapas de um pipeline de RAG? Explique brevemente cada uma das seguintes fases:
- Chunking
- Indexing
- Retrieval
- Reranking
- Geração (Generation)

# Exercício 2 – Prático: Chunking de Documentos

Implemente uma função que realiza o *chunking* de um documento de texto longo.

Requisitos:
- A função deve dividir o texto em pedaços (chunks) de tamanho configurável (ex.: 500 tokens ou 1000 caracteres).
- Deve permitir uma sobreposição (*overlap*) entre os chunks (ex.: 20% de sobreposição).
- O output esperado é uma lista de chunks.

Exemplo de uso esperado:
```python
chunks = chunk_text(long_text, chunk_size=500, overlap=100)
```

# Exercício 3 – Prático: Indexação Vetorial

Com os chunks criados no exercício anterior:

1. Gere embeddings para cada chunk usando um modelo de embeddings (ex.: `sentence-transformers`, OpenAI embeddings ou outro de sua escolha).
2. Crie um índice vetorial (FAISS, ChromaDB, Elasticsearch, etc.).
3. Implemente uma função para realizar buscas no índice, retornando os Top-K documentos mais similares a uma query.

Exemplo de uso esperado:
```python
results = search_index(query="Qual o impacto da inflação?", top_k=5)
```

# Exercício 4 – Prático: Recuperação de Contexto + Geração de Resposta

Implemente uma função que faça o seguinte:

1. Receba uma query do usuário.
2. Recupere os Top-K chunks mais relevantes do índice vetorial.
3. Monte um *prompt* contendo a query + os textos dos chunks recuperados.
4. Envie o prompt para um LLM (ex.: OpenAI, Mistral, Llama ou outro) para gerar uma resposta.

Exemplo de uso esperado:
```python
response = rag_pipeline(query="Explique o conceito de inflação")
print(response)
```

# Exercício 5 – Prático: Reranking

Implemente uma etapa de reranking para melhorar a qualidade dos documentos recuperados:

1. Após o retrieval inicial (ex.: Top-10), use um modelo de reranking (ex.: um `cross-encoder` da `sentence-transformers`) para reordenar os documentos com base na relevância para a query.
2. Compare as respostas geradas pelo RAG antes e depois do reranking.
3. Discuta brevemente se o reranking trouxe melhorias na qualidade da resposta.

Exemplo de uso esperado:
```python
ranked_results = rerank(query, retrieved_docs)
response = generate_answer(query, ranked_results)
```

## Import and Install
Todos os imports e %pip install que foram usados nos exercícios.

In [11]:
# Pip Install
%pip install sentence-transformers faiss-cpu --quiet
%pip install openai --quiet

# Set OpenAI API key 
import os
os.environ["OPENAI_API_KEY"] = "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"
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Resposta - Exercício 1

1. **O que é RAG (Retrieval-Augmented Generation) e qual problema ele resolve na geração de texto usando modelos de linguagem?**  
RAG é uma técnica que combina modelos de linguagem (LLMs) com mecanismos de busca em bases de dados externas. O objetivo é permitir que o modelo acesse informações atualizadas ou específicas durante a geração de texto, superando a limitação dos LLMs tradicionais, que só usam o conhecimento aprendido até o momento do treinamento. Assim, o RAG resolve o problema de respostas desatualizadas ou incompletas, tornando o modelo capaz de citar fontes e trazer dados recentes.

2. **Vantagens de utilizar RAG em vez de depender apenas do modelo base:**  
- Permite respostas mais precisas e atualizadas, pois consulta fontes externas.
- Reduz a alucinação (respostas inventadas) dos modelos.
- Possibilita citar fontes e justificar respostas.
- Facilita a adaptação para domínios específicos, usando bases de dados customizadas.

3. **Principais etapas de um pipeline de RAG:**
- **Chunking:** Dividir documentos em partes menores (chunks) para facilitar a busca e o processamento.
- **Indexing:** Criar um índice vetorial dos chunks, permitindo buscas rápidas por similaridade semântica.
- **Retrieval:** Buscar os chunks mais relevantes para uma consulta do usuário.
- **Reranking:** Reordenar os resultados recuperados, priorizando os mais relevantes para a query.
- **Geração (Generation):** Usar o modelo de linguagem para gerar uma resposta, combinando a query do usuário com os chunks recuperados.

## Resposta - Exercício 2
- A função `chunk_text` divide um texto longo em pedaços (chunks) de tamanho configurável (`chunk_size`).

- Permite uma sobreposição entre os chunks, controlada pelo parâmetro `overlap`.

- O output é uma lista de chunks, como solicitado.

- O exemplo de uso mostra como utilizar a função.

In [12]:
# Example long text
long_text = "Example of a long text that will be split into smaller chunks." * 100  # Simulating a long text

# Function to split a long text into chunks with overlap
def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    text_length = len(text)
    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)
        # Move to the next chunk, considering the overlap
        start += chunk_size - overlap
    return chunks

# Applying the function
chunks = chunk_text(long_text)

# Visualization of the result
print(f"Number of chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ---\n{chunk[:300]}...")  #mostra os primeiros 300 caracteres de cada parte


# Example usage:
# long_text = "..." 
# chunks = chunk_text(long_text, chunk_size=1000, overlap=200)
# print(f"Number of chunks: {len(chunks)}")
# print(chunks[0])

Number of chunks: 8

--- Chunk 1 ---
Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into small...

--- Chunk 2 ---
hunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into...

--- Chunk 3 ---
ller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be split into smaller chunks.Example of a long text that will be spli...

--- Chunk 4 ---


## Resposta - Exercício 3
- Gera embeddings para cada chunk usando um modelo de sentence transformer.

- Cria um índice FAISS para busca vetorial rápida.

- Implementa uma função para buscar os chunks mais similares a uma query.

- Retorna os top-k chunks mais similares e seus scores.

In [13]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Example: list of text chunks (replace with your actual chunks)
chunks = [
    "Inflation is the general increase in prices over time.",
    "Central banks use interest rates to control inflation.",
    "High inflation can erode purchasing power.",
    "Deflation is the opposite of inflation."
]

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each chunk
embeddings = model.encode(chunks)

# Create a FAISS index for fast similarity search
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Function to search the index and return top-k most similar chunks
def search_index(query, top_k=3):
    """
    Search the FAISS index for the most similar chunks to the query.
    Args:
        query (str): The search query.
        top_k (int): Number of top results to return.
    Returns:
        List of (chunk, score) tuples.
    """
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        results.append((chunks[idx], dist))
    return results

# Example usage:
results = search_index("What is inflation?", top_k=2)
print("Most similar results to the query:\n")
for i, (chunk, score) in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print(f"Chunk: {chunk}")
    print(f"Score (lower is more similar): {score:.4f}")
    print()

Most similar results to the query:

--- Result 1 ---
Chunk: Inflation is the general increase in prices over time.
Score (lower is more similar): 0.5827

--- Result 2 ---
Chunk: High inflation can erode purchasing power.
Score (lower is more similar): 0.6654



## Resposta - Exercício 4
- Receba uma query do usuário.
- Recupere os Top-K chunks mais relevantes do índice vetorial.
- Monte um *prompt* contendo a query + os textos dos chunks recuperados.
- Envie o prompt para um LLM (ex.: OpenAI, Mistral, Llama ou outro) para gerar uma resposta.

In [ ]:
# RAG pipeline function: Retrieve context and generate answer using OpenAI
import openai

def rag_pipeline(query, chunks, model_name="gpt-3.5-turbo", top_k=3):
    """
    Retrieves the top-k most relevant chunks and generates an answer with OpenAI LLM.
    Args:
        query (str): User question.
        chunks (list): List of text chunks.
        model_name (str): OpenAI model name.
        top_k (int): How many chunks to use as context.
    Returns:
        str: Answer generated by the model.
    """

    # Generate embeddings for the chunks and the query
    from sentence_transformers import SentenceTransformer
    import numpy as np
    model = SentenceTransformer('all-MiniLM-L6-v2')
    chunk_embs = model.encode(chunks)
    query_emb = model.encode([query])

    # Compute similarity (dot product)
    sims = np.dot(chunk_embs, query_emb[0])
    # Get indices of top_k most similar chunks
    top_indices = np.argsort(sims)[-top_k:][::-1]
    top_chunks = [chunks[i] for i in top_indices]

    # Build prompt with context
    context = "\n".join(top_chunks)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    # Generate answer using OpenAI (new API >=1.0.0)
    response = openai.resources.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()

# Example usage of the RAG pipeline
# Use the already defined 'chunks' variable from previous cells
test_query = "Explain the concept of inflation"


## Resposta - Exercício 5
- Após o retrieval inicial (ex.: Top-10), use um modelo de reranking (ex.: um `cross-encoder` da `sentence-transformers`) para reordenar os documentos com base na relevância para a query.
- Compare as respostas geradas pelo RAG antes e depois do reranking.
- Discuta brevemente se o reranking trouxe melhorias na qualidade da resposta.

In [25]:
# Reranking function using sentence-transformers cross-encoder
from sentence_transformers import CrossEncoder

def rerank(query, retrieved_docs, model_name='cross-encoder/ms-marco-MiniLM-L-6-v2'):
    """
    Reorders retrieved documents based on relevance to the query using a cross-encoder.
    Args:
        query (str): User query.
        retrieved_docs (list): List of retrieved texts.
        model_name (str): Cross-encoder model name.
    Returns:
        list: List of (doc, score) sorted by relevance.
    """
    cross_encoder = CrossEncoder(model_name)
    pairs = [[query, doc] for doc in retrieved_docs]
    scores = cross_encoder.predict(pairs)
    ranked = sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)
    return ranked

# Function to generate final answer using the reranked documents
import openai

def generate_answer(query, ranked_docs, model_name="gpt-3.5-turbo"):
    """
    Generates an answer using the reranked documents as context.
    Args:
        query (str): User query.
        ranked_docs (list): List of (doc, score) sorted by relevance.
        model_name (str): OpenAI model name.
    Returns:
        str: Answer generated by the model.
    """
    context = "\n".join([doc for doc, _ in ranked_docs])
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    response = openai.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=256
    )
    return response.choices[0].message.content.strip()

# Visualization of the pipeline with reranking (Exercise 5)

# Example of retrieved documents (replace with your actual chunks)
retrieved_docs = [
    "Inflation is the general increase in prices over time.",
    "Central banks use interest rates to control inflation.",
    "High inflation can erode purchasing power.",
    "Deflation is the opposite of inflation."
]

query = "Explain the concept of inflation"

# Reranking the documents
ranked_results = rerank(query, retrieved_docs)

# Visualization of results
print(f"Query: {query}\n")
print("Retrieved documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"--- Doc {i} ---\n{doc}\n")

print("Documents after reranking:")
for i, (doc, score) in enumerate(ranked_results, 1):
    print(f"--- Doc {i} (score={score:.4f}) ---\n{doc}\n")



# Exemplo de uso:
# retrieved_docs = ["Texto 1...", "Texto 2...", ...]
# ranked_results = rerank("Explique o conceito de inflação", retrieved_docs)
# resposta = generate_answer("Explique o conceito de inflação", ranked_results)
# print(resposta)

Query: Explain the concept of inflation

Retrieved documents:
--- Doc 1 ---
Inflation is the general increase in prices over time.

--- Doc 2 ---
Central banks use interest rates to control inflation.

--- Doc 3 ---
High inflation can erode purchasing power.

--- Doc 4 ---
Deflation is the opposite of inflation.

Documents after reranking:
--- Doc 1 (score=5.0835) ---
Inflation is the general increase in prices over time.

--- Doc 2 (score=-2.7351) ---
Deflation is the opposite of inflation.

--- Doc 3 (score=-6.8251) ---
High inflation can erode purchasing power.

--- Doc 4 (score=-9.9385) ---
Central banks use interest rates to control inflation.

